In [1476]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import holidays

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import BallTree
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import Pipeline

In [1477]:
df_train = pd.read_parquet(Path("../data") / "train.parquet")
df_test = pd.read_parquet(Path("../data") / "final_test.parquet")

X_train = df_train.drop(columns=['log_bike_count', 'bike_count'])
y_train = df_train['log_bike_count']
X_test = df_test

X_test.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 01:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
1,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 13:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
2,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 17:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
3,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 19:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
4,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 22:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429


In [1478]:
X_train.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429


In the following cell, we drop columns from the dataset:

In [1488]:
# def add_cyclical_date_features(X):
#     # Make sure your date_column is a datetime object
#     X['date'] = pd.to_datetime(X['date'])

#     # Day of the year
#     day_of_year = X['date'].dt.dayofyear
#     X['day_sin'] = np.sin(2 * np.pi * day_of_year / 365)
#     X['day_cos'] = np.cos(2 * np.pi * day_of_year / 365)

#     # Hour of the day
#     hour_of_day = X['date'].dt.hour
#     X['hour_sin'] = np.sin(2 * np.pi * hour_of_day / 24)
#     X['hour_cos'] = np.cos(2 * np.pi * hour_of_day / 24)

#     return X.drop(columns=['date'])

def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    # X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"]) #, "month", "day", "hour"

def _drop_init_cols(X):
    return X.drop(columns=['counter_name',
                           'coordinates',
                           'site_name',
                           'site_id',
                           'counter_technical_id',
                           'counter_installation_date',
                           ]) #'latitude', 'longitude'

def _merge_external_data(X):
    file_path = Path("../Notebooks") / "weather_v1.csv"
    df_ext = pd.read_csv(file_path, parse_dates=["date"])

    X = X.copy()
    X['date'] = X['date'].astype('datetime64[ns]')
    # When using merge_asof left frame need to be sorted
    X["orig_index"] = np.arange(X.shape[0])
    X = pd.merge_asof(
        X.sort_values("date"), df_ext[['date', 'pmer', 'tend', 'cod_tend', 'tend24',
                                       'dd', 'ff', 't', 'td', 'u', 'vv',  
                                       'n', 'pres', 'raf10', 'ww', 'nbas',
                                       'ht_neige', 'rr1', 'rr6',]].sort_values("date"), on="date"
    )  

    # Sort back to the original order
    X = X.sort_values("orig_index")
    del X["orig_index"]
    return X

def _add_holiday_column(X):
    years=[2020, 2021]
    fr_holidays = holidays.France(years=years)
    X = X.copy()

    def is_holiday(date):
        weekday = date.weekday()
        if weekday > 4 or date in fr_holidays:
            return 1
        else:
            return 0

    X['is_holiday'] = X['date'].apply(is_holiday)

    return X

def _merge_connected_roads(X):
    file_path = Path("../Notebooks") / "reseau_cyclable_v1.csv"
    df_cyclable_roads = pd.read_csv(file_path)

    # Convert lat/lon to radians for haversine distance calculation
    lat_lon_cyclable_roads = np.deg2rad(df_cyclable_roads[['latitude', 'longitude']].values)
    lat_lon_original = np.deg2rad(X[['latitude', 'longitude']].values)

    # Create a BallTree with cyclable roads' coordinates
    tree = BallTree(lat_lon_cyclable_roads, metric='haversine')

    # Define your search radius in meters and convert to radians (Earth radius is approximately 6371 km)
    radius = 100 / 6371000

    # Query the tree for roads within the radius for each bike traffic point
    indices = tree.query_radius(lat_lon_original, r=radius)

    # Copy the DataFrame to avoid modifying the original one
    X = X.copy()

    # Count the number of roads within the radius for each site and add to DataFrame
    X['number_of_connected_roads'] = [len(index) for index in indices]

    return X

def _merge_velib_info(X):
    file_path = Path("../Notebooks") / "info_velib_v1.csv"
    df_velib = pd.read_csv(file_path)

    # Convert lat/lon to radians for haversine distance calculation
    lat_lon_stations = np.deg2rad(df_velib[['latitude', 'longitude']].values)
    lat_lon_original = np.deg2rad(X[['latitude', 'longitude']].values)

    # Create a BallTree with station coordinates
    tree = BallTree(lat_lon_stations, metric='haversine')

    # Define your search radius in meters and convert to radians (Earth radius is approximately 6371 km)
    radius = 150 / 6371000  # Example radius of 500 meters

    # Query the tree for stations within the radius for each point in X
    indices = tree.query_radius(lat_lon_original, r=radius)

    X = X.copy() # copy the DataFrame to avoid modifying the original one

    # Calculate the sum of capacities for stations within the radius for each site in X
    X['total_nearby_station_capacity'] = [df_velib.iloc[index]['Capacité de la station'].sum() for index in indices]
    X['number_of_nearby_stations'] = [len(index) for index in indices]

    return X

def _process_column_id(X):
    X = X.copy() 
    
    def process_id(counter_id):
        parts = counter_id.split('-')
        # If the second part is numeric, use it; otherwise, use the first part
        return parts[1] if parts[1].isdigit() else parts[0]
    
    X['counter_id'] = X['counter_id'].apply(process_id).astype(str)

    X['longitude'] = X['longitude'].astype(str)
    X['latitude'] = X['latitude'].astype(str)

    # Concatenate 'longitude' and 'latitude' to 'counter_id'
    X['counter_id'] = X['counter_id'] + '_' + X['longitude'] + '_' + X['latitude']

    return X.drop(columns=['latitude', 'longitude'])

def _merge_fuel_index(X):
    file_path = Path("../Notebooks") / "fuel_index_v1.csv"
    df_fuel = pd.read_csv(file_path)
    
    X['date'] = pd.to_datetime(X['date'])
    X['year'] = X['date'].dt.year
    X['month'] = X['date'].dt.month

    X = X.copy()

    X = pd.merge(
        X, 
        df_fuel, 
        on=['year', 'month'],
        how='left'
    )

    return X.drop(['year', 'month'], axis=1)

# def _merge_covid_data(X):
#     file_path = Path("../Notebooks") / "covid_data_v1.csv"
#     df_lockdown = pd.read_csv(file_path)

#     # Ensure 'date' in X is a datetime object
#     X['date'] = pd.to_datetime(X['date'])

#     # Create 'year', 'month', and 'day' in X if they don't exist
#     X['year'] = X['date'].dt.year
#     X['month'] = X['date'].dt.month
#     X['day'] = X['date'].dt.day

#     X = X.copy()

#     # Merge the original data with the lockdown data
#     X = pd.merge(
#         X, 
#         df_lockdown, 
#         on=['year', 'month', 'day'],
#         how='left'
#     )

#     return X.drop(['year', 'month', 'day'], axis=1)

# def _add_total_traffic(X):
#     file_path = '../Notebooks/total_traffic_v1.csv'
#     df_total_traffic = pd.read_csv(file_path)

#     X['date'] = pd.to_datetime(X['date']) 
#     df_total_traffic['date'] = pd.to_datetime(df_total_traffic['date'])

#     X = X.copy()
#     # When using merge_asof left frame need to be sorted
#     X["orig_index"] = np.arange(X.shape[0])
#     X = pd.merge_asof(
#         X.sort_values("date"), df_total_traffic[['date', 'total_traffic']].sort_values("date"), on="date"
#     )

#     # Sort back to the original order
#     X = X.sort_values("orig_index")
#     del X["orig_index"]
#     return X


In [1489]:
# cyclical_date_transformer = FunctionTransformer(add_cyclical_date_features)

date_encoder = FunctionTransformer(_encode_dates)
is_holiday_col = FunctionTransformer(_add_holiday_column)
init_data_eng = FunctionTransformer(_drop_init_cols)
merge_external_data = FunctionTransformer(_merge_external_data, validate=False)
merge_connected_roads = FunctionTransformer(_merge_connected_roads)
merge_velib_info = FunctionTransformer(_merge_velib_info)
process_column_id = FunctionTransformer(_process_column_id)
merge_fuel_index = FunctionTransformer(_merge_fuel_index)
# merge_covid_data = FunctionTransformer(_merge_covid_data)
# add_total_traffic = FunctionTransformer(_add_total_traffic)

In the following cell, we do some feature engineering on our dataset:

In [1490]:
categorical_columns = ['counter_id']
#num_cols = 
one_hot = OneHotEncoder(handle_unknown='ignore')

def one_hot_encode_and_concat(X):
    one_hot_encoded_data = one_hot.fit_transform(X[categorical_columns])

    one_hot_encoded_df = pd.DataFrame(one_hot_encoded_data.toarray(), 
                                      columns=one_hot.get_feature_names_out(categorical_columns))

    X_dropped = X.drop(columns=categorical_columns)
    X_encoded = pd.concat([X_dropped.reset_index(drop=True), one_hot_encoded_df.reset_index(drop=True)], axis=1)

    return X_encoded

one_hot_transformer = FunctionTransformer(one_hot_encode_and_concat)

# def encode_counter_id(X):
#     if 'counter_id' in X.columns:
#         encoder = OrdinalEncoder()
#         X['counter_id'] = encoder.fit_transform(X[['counter_id']])
#     return X

# ordinal_encoder_transformer = FunctionTransformer(encode_counter_id)



In [1491]:
from xgboost import XGBRegressor

# regressor = RandomForestRegressor(
#     max_depth=20, 
#     n_estimators=200,
#     random_state=42
# )

regressor = XGBRegressor(
    max_depth=8, 
    n_estimators=300,
    learning_rate=0.18, 
    random_state=None
)

In [1492]:
pipeline = Pipeline([
    ("merging_ext_data", merge_external_data),
    ("adding_is_holiday_column", is_holiday_col),
    ("merging_connected_roads_data", merge_connected_roads),
    ("merging_velib_info", merge_velib_info),
    ("merging_fuel_index_info", merge_fuel_index),
    # ("merge_covid_data", merge_covid_data),
    # ("merge_total_traffic", add_total_traffic),
    ("encoding_dates", date_encoder),
    # ("cyclical_dates", cyclical_date_transformer),
    ("dropping_redundant_columns_in_initial_data", init_data_eng),
    ("processing_column_id", process_column_id),
    ("one_hot_encoding", one_hot_transformer),
    # ("ordinal_encoder", ordinal_encoder_transformer),
    # ("scaler", StandardScaler()),
    ("regressor", regressor)
])

In [1493]:
# X_train.to_csv("df_original.csv", index=False)

In [1324]:
# pipeline.fit_transform(X_train, y_train)

x = pipeline.fit_transform(X_train, y_train)
x_df = pd.DataFrame(x)

# x_df [x_df['site_name'] == 'Totem 73 boulevard de Sébastopol']



In [1325]:
x_df.shape

(496827, 61)

In [1326]:
x_df

,year,month,day,weekday,hour,counter_id_100044493_2.37376_48.86149,counter_id_100056046_2.31061_48.86282,counter_id_100056047_2.32003_48.86378,counter_id_100056223_2.310345_48.86284,counter_id_101007049_2.375429_48.846028,...,counter_id_105056336_2.35423_48.85013,counter_id_106056336_2.35423_48.85013,counter_id_109042374_2.27586_48.8484,counter_id_110042374_2.27586_48.8484,counter_id_353245971_2.30198_48.83977,counter_id_353245972_2.30198_48.83977,counter_id_353255859_2.333233_48.840801,counter_id_353255860_2.333233_48.840801,counter_id_353277233_2.32666_48.88529,counter_id_353277235_2.32666_48.88529
0,2020,9,1,1,2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020,9,1,1,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020,9,1,1,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020,9,1,1,15,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020,9,1,1,18,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496822,2021,9,9,3,6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
496823,2021,9,9,3,10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
496824,2021,9,9,3,15,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
496825,2021,9,9,3,22,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [1249]:
x_df['counter_id'].nunique()

56

In [1494]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('merging_ext_data',
                 FunctionTransformer(func=<function _merge_external_data at 0x0000023DAFD90180>)),
                ('adding_is_holiday_column',
                 FunctionTransformer(func=<function _add_holiday_column at 0x0000023DAFD919E0>)),
                ('merging_connected_roads_data',
                 FunctionTransformer(func=<function _merge_connected_roads at 0x0000023DAFD91620>)),
                ('merging_velib_inf...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.18,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=8, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=300, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [1336]:
importances = pipeline.named_steps['regressor'].feature_importances_
importances
# Create a DataFrame to display feature importances
# importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})
# importance_df = importance_df.sort_values(by='Importance', ascending=False)

# # Plot the feature importances
# plt.figure(figsize=(10, 6))
# plt.barh(importance_df['Feature'], importance_df['Importance'])
# plt.xlabel('Feature Importance')
# plt.ylabel('Feature')
# plt.title('XGBoost Regressor Feature Importances')
# plt.gca().invert_yaxis()
# plt.show()

array([0.0005226 , 0.00058446, 0.00045136, 0.00768238, 0.00125662,
       0.00323118, 0.0006106 , 0.00216893, 0.00083643, 0.00094805,
       0.00118917, 0.00124285, 0.01497126, 0.04195547, 0.06394488,
       0.0486289 , 0.00587882, 0.        , 0.00895645, 0.00251436,
       0.00257653, 0.03623594, 0.05334074, 0.01856647, 0.00092096,
       0.0115991 , 0.00390008, 0.00437626, 0.08742413, 0.01024474,
       0.00349134, 0.00451934, 0.00226507, 0.00090248, 0.01071067,
       0.00676583, 0.00286412, 0.0077104 , 0.00581813, 0.0034901 ,
       0.01920168, 0.00167457, 0.0372934 , 0.00249199, 0.03282342,
       0.00154541, 0.01750024, 0.04387255, 0.01128361, 0.00175734,
       0.00649656, 0.00550885, 0.00636642, 0.00315088, 0.01096424,
       0.0015703 , 0.0089499 , 0.03656341, 0.02912474, 0.0096618 ,
       0.04348949, 0.02954716, 0.00249006, 0.01554878, 0.02519668,
       0.00391911, 0.00073008, 0.03287507, 0.01465507, 0.00445119,
       0.00984446, 0.00651536, 0.00658937, 0.00205124, 0.00665

In [1495]:
y_pred = pipeline.predict(X_test)

In [1496]:
results = pd.DataFrame(
    dict(
        Id=np.arange(y_pred.shape[0]),
        log_bike_count=y_pred,
    )
)

results.to_csv("submission.csv", index=False)

In [1497]:
test = pd.read_csv("submission.csv")
test.head(10)

,Id,log_bike_count
0,0,0.234908
1,1,1.753227
2,2,2.189289
3,3,0.721036
4,4,0.729157
5,5,0.626252
6,6,0.713315
7,7,0.668682
8,8,0.582189
9,9,0.958133


## **<u>Train-test splitting and doing grid_search:</u>**

Because we are dealing with temporal data and our objective is to use historical data to predict future data, we define a new train test split.

In [1309]:
def train_test_split_temporal(X, y, delta_threshold="30 days"):
    
    cutoff_date = X["date"].max() - pd.Timedelta(delta_threshold)
    mask = (X["date"] <= cutoff_date)
    X_train, X_valid = X.loc[mask], X.loc[~mask]
    y_train, y_valid = y[mask], y[~mask]

    return X_train, y_train, X_valid, y_valid

In [1310]:
X_train_loc, y_train_loc, X_test_loc, y_test_loc = train_test_split_temporal(X_train, y_train)

In [1311]:
pipeline.fit(X_train_loc, y_train_loc)

Pipeline(steps=[('merging_ext_data',
                 FunctionTransformer(func=<function _merge_external_data at 0x0000023D82445E40>)),
                ('adding_is_holiday_column',
                 FunctionTransformer(func=<function _add_holiday_column at 0x0000023DAFD93A60>)),
                ('merging_connected_roads_data',
                 FunctionTransformer(func=<function _merge_connected_roads at 0x0000023DAFD918A0>)),
                ('merging_velib_inf...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=8, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=225, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [1312]:
y_pred = pipeline.predict(X_test_loc)

In [1313]:
mse = mean_squared_error(y_test_loc, y_pred)

# Calculate root mean squared error
rmse = np.sqrt(mse)

print(f"RMSE: {rmse}")

RMSE: 0.49850435451031905


In [60]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

cv = TimeSeriesSplit(n_splits=6)

# When using a scorer in scikit-learn it always needs to be better when smaller, hence the minus sign.
scores = cross_val_score(
    pipeline, X_train_loc, y_train_loc, cv=cv, scoring="neg_root_mean_squared_error"
)
print("RMSE: ", scores)
print(f"RMSE (all folds): {-scores.mean():.3} ± {(-scores).std():.3}")

KeyboardInterrupt: 

In [1314]:
# Using GridSearch

# param_grid = {
#     'regressor__max_depth': [15, 20, 25],
#     'regressor__n_estimators': [100, 200]
# }

param_grid = {
    'regressor__max_depth': [8, 10],
    'regressor__n_estimators': [200, 300],
    'regressor__learning_rate': [0.08, 0.1, 0.12]
}

grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1
)

grid_search.fit(X_train_loc, y_train_loc)

print("Best Parameters Found:")
print(grid_search.best_params_)
print('\n')

print("Best Score: ", grid_search.best_score_)
print("Best Parameters: ", grid_search.best_params_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best Parameters Found:
{'regressor__learning_rate': 0.08, 'regressor__max_depth': 8, 'regressor__n_estimators': 300}


Best Score:  -1.4228078507166595
Best Parameters:  {'regressor__learning_rate': 0.08, 'regressor__max_depth': 8, 'regressor__n_estimators': 300}
